In [7]:
!pip install opencv-python


  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
    --------------------------------------- 0.5/39.0 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.8/39.0 MB 1.2 MB/s eta 0:00:31
    --------------------------------------- 0.8/39.0 MB 1.2 MB/s eta 0:00:31
   - -------------------------------------- 1.0/39.0 MB 1.1 MB/s eta 0:00:35
   - -------------------------------------- 1.3/39.0 MB 1.2 MB/s eta 0:00:33
   - -------------------------------------- 1.8/39.0 MB 1.3 MB/s eta 0:00:29
   -- ------------------------------------- 2.1/39.0 MB 1.3 MB/s eta 0:00:28
   -- ------------------------------------- 2.6/39.0 MB 1.4 MB/s eta 0:00:27
   -- ------------------------------------- 2.6/39.0 MB 1.4 MB/s eta 0:00:27
   --- ------------------------------------ 3.1/39.0 MB 1.4 MB/s eta 0:00:26
   --- -------------

In [2]:
import cv2
import numpy as np
import time

print("OpenCV Version:", cv2.__version__)

# Starts camera
cap = cv2.VideoCapture(0)
time.sleep(1)

# Captures background
print("[INFO] Capturing background... Please move out of the frame.")
background = None
for _ in range(50):
    ret, background = cap.read()
    if not ret:
        continue

background = np.flip(background, axis=1)
print("[INFO] Background captured successfully.")

# Fixed GREEN color range in HSV
lower_green = np.array([35, 60, 40])
upper_green = np.array([85, 255, 255])

kernel = np.ones((3, 3), np.uint8)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = np.flip(frame, axis=1)

    # Converts BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Creates green color mask
    cloak_mask = cv2.inRange(hsv, lower_green, upper_green)

    # Improves mask quality
    cloak_mask = cv2.morphologyEx(cloak_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    cloak_mask = cv2.dilate(cloak_mask, kernel, iterations=1)

    inverse_mask = cv2.bitwise_not(cloak_mask)

    # Extracts cloak area from background
    cloak_region = cv2.bitwise_and(background, background, mask=cloak_mask)

    # Extract visible area from current frame
    normal_region = cv2.bitwise_and(frame, frame, mask=inverse_mask)

    # Final combined output
    final_output = cv2.add(cloak_region, normal_region)

    cv2.imshow("Invisibility Cloak - Green Only", final_output)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


OpenCV Version: 4.12.0
[INFO] Capturing background... Please move out of the frame.
[INFO] Background captured successfully.
